In [4]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series
from numpy import nan as NA

In [2]:
Series(['abc','bcd']).str.replace('a','A')

0    Abc
1    bcd
dtype: object

In [5]:
df1 = DataFrame({'a':[10,NA,9,1],
                'b':['abc','bcd','efc','add']})

In [6]:
df1.replace(10,100)

,a,b
0,100.0,abc
1,NaN,bcd
2,9.0,efc
3,1.0,add


In [7]:
df1.fillna(0)

,a,b
0,10.0,abc
1,0.0,bcd
2,9.0,efc
3,1.0,add


### 멀티인덱스

In [10]:
df1 = DataFrame(np.arange(1,9).reshape(4,2))
df1.index = [['A','A','B','B'],['a','b','a','b']]
df1.columns = ['col1','col2']
df1.index.names=['상위인덱스','하위인덱스']
df1

col1  col2
상위인덱스 하위인덱스            
A     a         1     2
      b         3     4
B     a         5     6
      b         7     8

In [21]:
df3 = pd.read_csv('multi_index.csv', engine='python')
df3

,Unnamed: 0,Unnamed: 1,냉장고,Unnamed: 3,TV,Unnamed: 5
0,NaN,NaN,price,qty,price,qty
1,seoul,A,250,50,300,40
2,NaN,B,255,49,305,41
3,incheon,A,240,60,290,50
4,NaN,B,245,70,280,60


In [22]:
# step 1) 첫번째 컬럼 NA 치환
df3.iloc[:,0] = df3.iloc[:,0].fillna(method='ffill')
df3

,Unnamed: 0,Unnamed: 1,냉장고,Unnamed: 3,TV,Unnamed: 5
0,NaN,NaN,price,qty,price,qty
1,seoul,A,250,50,300,40
2,seoul,B,255,49,305,41
3,incheon,A,240,60,290,50
4,incheon,B,245,70,280,60


In [ ]:
# step 2) 첫번째 두번째 컬럼 머리 인덱스 설정
df3 = df3.set_index(['Unnamed: 0','Unnamed: 1'])

In [28]:
df3

냉장고 Unnamed: 3     TV Unnamed: 5
Unnamed: 0 Unnamed: 1                                    
NaN        NaN         price        qty  price        qty
seoul      A             250         50    300         40
           B             255         49    305         41
incheon    A             240         60    290         50
           B             245         70    280         60

In [30]:
# step 3) 컬럼값 변경 => 냉장고 냉장고 TV Tv 

f1 = lambda x : np.where('Unnamed' in x, NA, x)

In [31]:
f1 = df3.columns.map(lambda x : NA if 'Unnamed' in x else x )

In [34]:
df3.columns = Series(f1).fillna(method='ffill')
df3

냉장고  냉장고     TV   TV
Unnamed: 0 Unnamed: 1                        
NaN        NaN         price  qty  price  qty
seoul      A             250   50    300   40
           B             255   49    305   41
incheon    A             240   60    290   50
           B             245   70    280   60

In [ ]:
# step 4 ) 멀티 컬럼 설정
df3.columns = [df3.columns, df3.iloc[0,:]]

In [40]:
df3

냉장고          TV     
(nan, nan)             price  qty  price  qty
Unnamed: 0 Unnamed: 1                        
NaN        NaN         price  qty  price  qty
seoul      A             250   50    300   40
           B             255   49    305   41
incheon    A             240   60    290   50
           B             245   70    280   60

In [ ]:
df3 = df3.drop(NA, axis=0,level=0)

In [46]:
df3.index.names = ['지점','지역']
df3

냉장고        TV    
(nan, nan) price qty price qty
지점      지역                    
seoul   A    250  50   300  40
        B    255  49   305  41
incheon A    240  60   290  50
        B    245  70   280  60

In [47]:
df3.columns.names=['품목','구분']
df3

품목           냉장고        TV    
구분         price qty price qty
지점      지역                    
seoul   A    250  50   300  40
        B    255  49   305  41
incheon A    240  60   290  50
        B    245  70   280  60

In [48]:
df3.xs('price',      # 이름
      axis=1,        # 색인방향(index:0, columns:!)
      level=1)       # 인덱스 level(상위:0, 하위:!)

품목          냉장고   TV
지점      지역          
seoul   A   250  300
        B   255  305
incheon A   240  290
        B   245  280

In [51]:
# ex ) A 지점의 price 선택
df3.xs('price', axis=1, level=1).xs('A',axis=0,level=1)

품목,냉장고,TV
지점,,
seoul,250,300
incheon,240,290


In [58]:
# ex ) 냉장고의 price, TV의 qty
df3.loc[:,[('냉장고','price'),('TV','qty')]]

품목           냉장고  TV
구분         price qty
지점      지역          
seoul   A    250  40
        B    255  41
incheon A    240  50
        B    245  60